In [51]:
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.datasets
import sklearn.metrics
import autosklearn.regression
import autosklearn.classification

train_name = 'GPY_with_100LH_20IT.csv'
model_name = 'GPY_with_100LH_20IT'

# auto sklearn regression


In [52]:
data_train=pd.read_csv(train_name)  

data_test=pd.read_csv("testdata_2500.csv")
print(data_train.shape,data_test.shape)
data_train.head(5)

(120, 9) (2500, 9)


,Ym,Pr,L,W,F1,F2,F3,F4,Displacement
0,393846.956325,0.283438,9785.064860,1989.719699,4.550465e+06,2.048921e+06,6.508967e+06,1.520737e+06,2.856307
1,258708.164914,0.278237,8689.675029,1916.843283,7.420939e+06,4.025130e+06,8.048466e+06,4.781460e+06,6.239527
2,367528.309421,0.448074,8656.216279,1824.658729,4.308446e+06,2.250449e+06,1.192787e+06,2.206634e+06,3.185127
3,280617.324465,0.116326,8273.875980,1513.627456,3.886773e+06,2.507154e+06,6.123143e+06,4.055736e+06,6.806090
4,144771.007364,0.352229,9322.079241,1085.685535,5.280718e+06,2.937048e+06,4.386006e+06,2.791044e+06,90.872201


In [53]:
X_train=data_train.drop(columns=[" Displacement"])
y_train=data_train[" Displacement"]

X_test=data_test.drop(columns=[" Displacement"])
y_test=data_test[" Displacement"]
X_test,y_test

(                 Ym        Pr            L            W            F1  \
 0      94395.485829  0.408768  8499.583680  1608.727367  1.029454e+06   
 1      84278.878309  0.231091  8788.419289  1175.258742  5.345039e+06   
 2     149552.529744  0.437990  8189.447620  1794.238778  4.589577e+06   
 3     190093.263121  0.295797  8212.301835  1936.807105  7.972106e+06   
 4     104785.791189  0.159446  9576.520014  1241.439274  2.022164e+06   
 ...             ...       ...          ...          ...           ...   
 2495  380701.056597  0.381405  8822.413422  1342.425112  5.613384e+06   
 2496  454265.280975  0.113959  8503.386193  1875.658788  9.362209e+06   
 2497  132188.668823  0.108465  9593.618716  1540.149936  4.534234e+06   
 2498   69032.490830  0.326947  9684.017646  1574.082671  3.061684e+06   
 2499  137123.343012  0.343410  8457.361162  1033.939568  1.066780e+06   
 
                 F2            F3            F4  
 0     2.754024e+06  6.663235e+06  3.989766e+06  
 1     1.6

In [54]:
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=150,
    per_run_time_limit=15,
    initial_configurations_via_metalearning=25,
    memory_limit=3072,
    resampling_strategy='cv',
    resampling_strategy_arguments={'folds':5},
    tmp_folder="/tmp/autosklearn_regression_example_tmp2",
    n_jobs=4
)
automl.fit(X_train, y_train, dataset_name=model_name) 

AutoSklearnRegressor(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                     n_jobs=4, per_run_time_limit=15, resampling_strategy='cv',
                     resampling_strategy_arguments={'folds': 5},
                     time_left_for_this_task=150,
                     tmp_folder='/tmp/autosklearn_regression_example_tmp2')

In [55]:
# check the general information of the auto-sklearn setting
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: GPY_with_100LH_20IT
  Metric: r2
  Best validation score: 0.652211
  Number of target algorithm runs: 59
  Number of successful target algorithm runs: 51
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0



In [56]:
print(automl.leaderboard(detailed=True))

          rank  ensemble_weight              type      cost   duration  \
model_id                                                                 
45           1             0.86               mlp  0.370901   2.423119   
5            2             0.14  gaussian_process  1.083231  12.891873   

          config_id  train_loss  seed    start_time      end_time  budget  \
model_id                                                                    
45               44    0.048093     0  1.686751e+09  1.686751e+09     0.0   
5                 4    0.006793     0  1.686750e+09  1.686750e+09     0.0   

                      status data_preprocessors      feature_preprocessors  \
model_id                                                                     
45        StatusType.SUCCESS                 []  [select_rates_regression]   
5         StatusType.SUCCESS                 []               [polynomial]   

         balancing_strategy           config_origin  
model_id                   

In [57]:
# save the model 
import pickle

filename = 'automl_model_{}.sav'.format(model_name) 
pickle.dump(automl, open(filename, 'wb'))

## Test the model

In [58]:
# reload the model
import pickle
filename = filename

loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.9004851807730556


In [59]:
print(automl.show_models())

{5: {'model_id': 5, 'rank': 1, 'cost': 1.0832308022110435, 'ensemble_weight': 0.14, 'voting_model': VotingRegressor(estimators=None), 'estimators': [{'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f3fcf520700>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f402342edd0>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f402342c730>, 'sklearn_regressor': GaussianProcessRegressor(alpha=0.035440248140559884,
                         kernel=RBF(length_scale=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
                         n_restarts_optimizer=10, normalize_y=True,
                         random_state=1)}, {'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f3fcf4c6d40>, 'f

In [60]:
# evaluate the best model
from sklearn.metrics import mean_absolute_error

y_pred = automl.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Test MAE: %.3f" % mae)
train_predictions = automl.predict(X_train)
print("Train R2 score:", sklearn.metrics.r2_score(y_train, train_predictions))
test_predictions = automl.predict(X_test)
print("Test R2 score:", sklearn.metrics.r2_score(y_test, test_predictions))

Test MAE: 4.483
Train R2 score: 0.9705467097338247
Test R2 score: 0.9004851807730556


In [11]:
# plt.scatter(test_predictions, y_test, label="Train samples", c="#d95f02")
# # plt.scatter(test_predictions, y_test, label="Test samples", c="#7570b3")
# plt.xlabel("Predicted value")
# plt.ylabel("True value")
# plt.legend()
# plt.plot([50, 350], [50, 350], c="k", zorder=0)
# plt.xlim([50, 350])
# plt.ylim([50, 350])
# plt.tight_layout()
# plt.show()

automl_model_LH: 
Test MAE: 4.079
Train R2 score: 0.9805968098049971
Test R2 score: 0.9078082766355309

automl_model_LH_with_320.sav
Test MAE: 3.769
Train R2 score: 0.9857039960928277
Test R2 score: 0.9318146351400987

automl_model_LH_with_220.sav
Test MAE: 5.810
Train R2 score: 0.9027895491232791
Test R2 score: 0.9021625496275776

automl_model_LH_with_120.sav
Test MAE: 4.445
Train R2 score: 0.9935270882953583
Test R2 score: 0.8207560708756729

automl_model_gpy
Test MAE: 3.815
Train R2 score: 0.9617766630544647
Test R2 score: 0.9526885191050858

automl_model_GPY_with_100LH_220IT.sav
Test MAE: 4.686
Train R2 score: 0.9697162292435806
Test R2 score: 0.9214437040707228

automl_model_GPY_with_100LH_120IT.sav
Test MAE: 4.285
Train R2 score: 0.9548977936924598
Test R2 score: 0.9393131452314745

automl_model_GPY_with_100LH_20IT.sav
Test MAE: 4.483
Train R2 score: 0.9705467097338247
Test R2 score: 0.9004851807730556